# imdb movies recommendation system 

in this project we choosed hybrid aproach (using collabrative and content base together) in order to design a recommendation system based on user input movies 

In [ ]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
m_df = pd.read_csv('movies_p.csv')
r_df = pd.read_csv('ratings_p.csv')


In [3]:
m_df.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


at first we  make a recommandation list based with pandas datafarme with  collabrative based solutions 

In [4]:

m_df['year'] = m_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
m_df['year'] = m_df.year.str.extract('(\d\d\d\d)' ,expand = False)
m_df['title'] = m_df.title.str.replace('(\(\d\d\d\d\))', '')
m_df['title'] = m_df['title'].apply(lambda x: x.strip())

C:\Users\sina\AppData\Local\Temp\ipykernel_7200\2647627890.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  m_df['title'] = m_df.title.str.replace('(\(\d\d\d\d\))', '')


In [5]:
m_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [6]:
m_df = m_df.drop('genres',1)

C:\Users\sina\AppData\Local\Temp\ipykernel_7200\1789542479.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  m_df = m_df.drop('genres',1)


In [7]:
r_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [9]:
 inId = m_df[m_df['title'].isin(inputMovies['title']).tolist()]
inputMovies = pd.merge (inId,inputMovies)
inputMovies=inputMovies.drop('year', 1)
inputMovies

C:\Users\sina\AppData\Local\Temp\ipykernel_7200\1946031123.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies=inputMovies.drop('year', 1)


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [10]:
subrating = r_df[r_df['movieId'].isin(inputMovies['movieId'].tolist())]
subrating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
16,1,296,3.0,964982967
320,4,296,1.0,945173350
422,4,1968,4.0,986934786
516,5,1,4.0,847434962
...,...,...,...,...
99510,609,296,4.0,847220802
99534,610,1,5.0,1479542900
99552,610,296,5.0,1479545817
99636,610,1274,5.0,1493846688


In [11]:
#finding the users who have saw the same movies 
usergr = subrating.groupby(['userId'])
usergr.get_group(91)

,userId,movieId,rating,timestamp
14121,91,1,4.0,1112713037
14122,91,2,3.0,1112713392
14173,91,296,4.5,1112711264
14316,91,1274,5.0,1112713057
14383,91,1968,3.0,1112713409


In [12]:
#finding user who have saw all of the five movies that has been entered 
usergr = sorted(usergr, key = lambda x: len(x[1]),reverse=True)

In [13]:
usergr = usergr [0:100]

In [14]:
#choosing the best similaryties base on pearson distance 
pearsdict = {}
for name , group in usergr:
    group = group.sort_values(by='movieId')
    inputMovies= inputMovies.sort_values(by='movieId')
    nRatings = len(group)
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    if Sxx != 0 and Syy != 0:
        pearsdict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsdict[name] = 0

In [15]:
pearsdict.items()

dict_items([(91, 0.43852900965351443), (177, 0.0), (219, 0.45124262819713973), (274, 0.716114874039432), (298, 0.9592712306918567), (414, 0.9376144618769914), (474, 0.11720180773462392), (477, 0.4385290096535153), (480, 0.7844645405527362), (483, 0.08006407690254357), (599, 0.7666866491579839), (608, 0.920736884379251), (50, 0.15713484026367722), (57, -0.7385489458759964), (68, 0.0), (103, 0.5222329678670935), (135, 0.8703882797784892), (182, 0.9428090415820635), (202, 0.5222329678670935), (217, 0.30151134457776363), (226, 0.9438798074485389), (288, 0.6005325641789633), (307, 0.9655810287305759), (318, 0.44486512077567225), (322, 0.5057805388588731), (330, 0.9035942578600878), (357, 0.5606119105813882), (434, 0.9864036607532465), (448, 0.30151134457776363), (469, 0.8164965809277261), (561, 0.5222329678670935), (600, 0.18442777839082938), (606, 0.9146591207600472), (610, -0.47140452079103173), (18, 1.0), (19, -0.5), (21, 0), (45, 0.5000000000000009), (63, -0.4999999999999982), (64, 0.0)

In [16]:
pears_df = pd.DataFrame.from_dict (pearsdict,orient='index')
pears_df.columns = ['similarity']
pears_df['userId'] = pears_df.index
pears_df.index = range(len(pears_df))
pears_df.head()

,similarity,userId
0,0.438529,91
1,0.000000,177
2,0.451243,219
3,0.716115,274
4,0.959271,298


In [17]:
topsim = pears_df.sort_values(by = 'similarity' , ascending = False)[0:50]
topsim.head()

,similarity,userId
43,1.0,132
34,1.0,18
63,1.0,305
82,1.0,489
86,1.0,525


In [18]:
toprating = topsim.merge(r_df , left_on = 'userId' , right_on = 'userId' , how = 'inner' )
toprating.head()

,similarity,userId,movieId,rating,timestamp
0,1.0,132,1,2.0,1157921785
1,1.0,132,17,3.0,1157922698
2,1.0,132,29,2.0,1157924165
3,1.0,132,32,3.0,1329983726
4,1.0,132,34,1.5,1157921395


In [19]:
toprating ['weightrate'] = toprating ['similarity'] * toprating['rating']
toprating.head(1)


,similarity,userId,movieId,rating,timestamp,weightrate
0,1.0,132,1,2.0,1157921785,2.0


In [20]:
temptoprating = toprating.groupby('movieId').sum()[['similarity','weightrate']]
temptoprating.columns = ['simindex','weightindex']
temptoprating.head()

,simindex,weightindex
movieId,,
1,36.354096,133.167946
2,31.005292,94.904257
3,8.783859,26.381456
4,0.866025,1.732051
5,7.165336,19.775255


In [21]:
recomcollab_df = pd.DataFrame()
recomcollab_df['recom_value'] = temptoprating['weightindex']/temptoprating['simindex']
recomcollab_df['movieId'] = temptoprating.index
recomcollab_df = recomcollab_df.sort_values (by = 'recom_value' , ascending = False)
recomcollab_df.head()

,recom_value,movieId
movieId,,
3310,5.0,3310
7579,5.0,7579
905,5.0,905
1211,5.0,1211
140627,5.0,140627


hear we use content based solution to filter the first recommand list 

In [22]:
nm_df = pd.read_csv('movies_p.csv')
recomcollab_df = recomcollab_df[0:500]

In [23]:
finrec = nm_df.loc[nm_df['movieId'].isin(recomcollab_df['movieId'].tolist())]
finrec.head()

,movieId,title,genres
15,16,Casino (1995),Crime|Drama
31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
46,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
76,85,Angels and Insects (1995),Drama|Romance
83,94,Beautiful Girls (1996),Comedy|Drama|Romance


In [24]:
nm_df['year'] = nm_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
nm_df['year'] = nm_df.year.str.extract('(\d\d\d\d)',expand=False)
nm_df['title'] = nm_df.title.str.replace('(\(\d\d\d\d\))', '')
nm_df['title'] = nm_df['title'].apply(lambda x: x.strip())
nm_df.head()

C:\Users\sina\AppData\Local\Temp\ipykernel_7200\1405040837.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  nm_df['title'] = nm_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [25]:
nm_df['genres'] = nm_df.genres.str.split('|')
nm_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [26]:
moviegen = nm_df.copy()
for index , row in nm_df.iterrows() : 
    for genres in row['genres'] : 
        moviegen.at[index,genres] = 1 
moviegen = moviegen.fillna(0)   
moviegen.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
r_df = r_df.drop('timestamp' , 1)
r_df.head()

C:\Users\sina\AppData\Local\Temp\ipykernel_7200\818986462.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  r_df = r_df.drop('timestamp' , 1)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [28]:
inputId = nm_df[nm_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
inputMovies

C:\Users\sina\AppData\Local\Temp\ipykernel_7200\4286699816.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
C:\Users\sina\AppData\Local\Temp\ipykernel_7200\4286699816.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('genres', 1).drop('year', 1)


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [29]:
userMovies = moviegen[moviegen['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
973,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1445,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
userMovies = userMovies.reset_index(drop=True)
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

C:\Users\sina\AppData\Local\Temp\ipykernel_7200\638483878.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\sina\AppData\Local\Temp\ipykernel_7200\638483878.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\sina\AppData\Local\Temp\ipykernel_7200\638483878.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\sina\AppData\Local\Temp\ipykernel_7200\638483878.py:2: FutureWarning: In a future version of pandas all

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [32]:
genreTable = moviegen.set_index(moviegen['movieId'])
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

C:\Users\sina\AppData\Local\Temp\ipykernel_7200\1468165150.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\sina\AppData\Local\Temp\ipykernel_7200\1468165150.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\sina\AppData\Local\Temp\ipykernel_7200\1468165150.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\sina\AppData\Local\Temp\ipykernel_7200\1468165150.py:2: FutureWarning: In a future version of pandas all argumen

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
#Multiply the genres by the weights and then take the weighted average
finalrec = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
finalrec.columns = ['collab_rate']
finalrec.head()

movieId
1    0.594406
2    0.293706
3    0.188811
4    0.328671
5    0.188811
dtype: float64

In [34]:
finalrec = finalrec.sort_values(ascending=False)
finalrec.head()

movieId
134853    0.734266
148775    0.685315
117646    0.678322
6902      0.678322
81132     0.671329
dtype: float64

In [35]:
recom = nm_df.loc[nm_df['movieId'].isin(finalrec.head(100).keys())]
recom

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
19,20,Money Train,"[Action, Comedy, Crime, Drama, Thriller]",1995
118,145,Bad Boys,"[Action, Comedy, Crime, Drama, Thriller]",1995
478,546,Super Mario Bros.,"[Action, Adventure, Children, Comedy, Fantasy,...",1993
559,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
...,...,...,...,...
9223,152081,Zootopia,"[Action, Adventure, Animation, Children, Comedy]",2016
9282,157865,Ratchet & Clank,"[Action, Adventure, Animation, Children, Comed...",2016
9358,161594,Kingsglaive: Final Fantasy XV,"[Action, Adventure, Animation, Drama, Fantasy,...",2016
9394,164226,Maximum Ride,"[Action, Adventure, Comedy, Fantasy, Sci-Fi, T...",2016


the final list of recommanded movie is hear whitch  has been filterd by content based approach 

In [68]:
moviesRecommanded = finrec.loc[finrec['movieId'].isin(finalrec.head(500).keys())]
moviesRecommanded = moviesRecommanded.drop(moviesRecommanded[moviesRecommanded['movieId'].isin(inputMovies['movieId'])].index)
moviesRecommanded.head(20)

,movieId,title,genres
585,720,Wallace & Gromit: The Best of Aardman Animatio...,Adventure|Animation|Comedy
868,1148,Wallace & Gromit: The Wrong Trousers (1993),Animation|Children|Comedy|Crime
899,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
916,1215,Army of Darkness (1993),Action|Adventure|Comedy|Fantasy|Horror
924,1223,"Grand Day Out with Wallace and Gromit, A (1989)",Adventure|Animation|Children|Comedy|Sci-Fi
1375,1884,Fear and Loathing in Las Vegas (1998),Adventure|Comedy|Drama
2077,2761,"Iron Giant, The (1999)",Adventure|Animation|Children|Drama|Sci-Fi
2260,3000,Princess Mononoke (Mononoke-hime) (1997),Action|Adventure|Animation|Drama|Fantasy
2350,3108,"Fisher King, The (1991)",Comedy|Drama|Fantasy|Romance
2453,3266,Man Bites Dog (C'est arrivé près de chez vous)...,Comedy|Crime|Drama|Thriller
